In [50]:
from datetime import datetime,timedelta

In [51]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'

resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
tr_list = soup.select('table.fixed tbody tr')
tr_list

[<tr>
 <td class="fb">2020</td>
 <td>05월 28일</td>
 <td>0.50</td>
 </tr>, <tr>
 <td class="fb">2020</td>
 <td>03월 17일</td>
 <td>0.75</td>
 </tr>, <tr>
 <td class="fb">2019</td>
 <td>10월 16일</td>
 <td>1.25</td>
 </tr>, <tr>
 <td class="fb">2019</td>
 <td>07월 18일</td>
 <td>1.50</td>
 </tr>, <tr>
 <td class="fb">2018</td>
 <td>11월 30일</td>
 <td>1.75</td>
 </tr>, <tr>
 <td class="fb">2017</td>
 <td>11월 30일</td>
 <td>1.50</td>
 </tr>, <tr>
 <td class="fb">2016</td>
 <td>06월 09일</td>
 <td>1.25</td>
 </tr>, <tr>
 <td class="fb">2015</td>
 <td>06월 11일</td>
 <td>1.50</td>
 </tr>, <tr>
 <td class="fb">2015</td>
 <td>03월 12일</td>
 <td>1.75</td>
 </tr>, <tr>
 <td class="fb">2014</td>
 <td>10월 15일</td>
 <td>2.00</td>
 </tr>, <tr>
 <td class="fb">2014</td>
 <td>08월 14일</td>
 <td>2.25</td>
 </tr>, <tr>
 <td class="fb">2013</td>
 <td>05월 09일</td>
 <td>2.50</td>
 </tr>, <tr>
 <td class="fb">2012</td>
 <td>10월 11일</td>
 <td>2.75</td>
 </tr>, <tr>
 <td class="fb">2012</td>
 <td>07월 12일</td>
 <td>3.00</td>

In [52]:
from datetime import date
result = []
for tr in tr_list[1:]:
    a = tr.text.split('\n')
    time = date(int(a[1]), int(a[2][0:2]), int(a[2][4:6])) # year, month, day
    base_rate = a[3]
    
    base_dict = {
        'date' : time,
        'base_rate' : base_rate
    }
    result.append(base_dict)

In [53]:
import pandas as pd
df = pd.DataFrame(result)
df = df.set_index('date')
df

,base_rate
date,
2020-03-17,0.75
2019-10-16,1.25
2019-07-18,1.50
2018-11-30,1.75
2017-11-30,1.50
2016-06-09,1.25
2015-06-11,1.50
2015-03-12,1.75
2014-10-15,2.00


In [54]:
# df.to_csv('기준금리')

In [55]:
fulldate = pd.DataFrame(columns = ['date'])
start = date(2005, 4, 1)
end = date(2017, 12, 31)
for n in range(int((end - start).days) + 1):
    fulldate.loc[n, 'date'] = start + timedelta(n)
fulldate

,date
0,2005-04-01
1,2005-04-02
2,2005-04-03
3,2005-04-04
4,2005-04-05
...,...
4653,2017-12-27
4654,2017-12-28
4655,2017-12-29
4656,2017-12-30


In [56]:
final_rates = pd.merge(fulldate,df, on = 'date',how='left')
# final_rates[''] = final_rates['price'].fillna(method='ffill')
final_rates['base_rate']=final_rates['base_rate'].fillna(method='bfill') 
final_rates['base_rate']=final_rates['base_rate'].fillna(method='ffill') 
final_rates

,date,base_rate
0,2005-04-01,3.50
1,2005-04-02,3.50
2,2005-04-03,3.50
3,2005-04-04,3.50
4,2005-04-05,3.50
...,...,...
4653,2017-12-27,1.50
4654,2017-12-28,1.50
4655,2017-12-29,1.50
4656,2017-12-30,1.50


In [57]:
final_rates.to_csv('전체기간_기준금리')